In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from keras.preprocessing import image

In [2]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_preprocess

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as inception_resnet_v2_preprocess

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa

In [4]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                 "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

In [5]:
width = 299
height = 299

In [6]:
# Path to test images
test_dir = '../input/plant-seedlings-classification/'
test_dir_seg = '../input/plant-seedling-segmented/plant-seedling-segmented/'

# Path to saved models
model_dir = '../input/plant-seedling-models/'

In [7]:
def create_datagen(preprocess_fn):
    return ImageDataGenerator(preprocessing_function=preprocess_fn,
                             )

In [8]:
def create_dataloader(datagen, test_dir_main, test):
    dataloader = datagen.flow_from_directory(
    directory=test_dir_main,
    classes=[test],
    target_size=(width, height),
    batch_size=1,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical')
    return dataloader


In [9]:
def create_xception():
    model_main = Xception(weights='imagenet', input_shape=(width, height, 3), include_top=False)
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)

    dropout_1 = tf.keras.layers.Dropout(0.2)(layer)
    model_dense2 = tf.keras.layers.Dense(256, activation = 'selu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.2)(model_dense2)
    model_dense3 = tf.keras.layers.Dense(128, activation='selu')(dropout_2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(model_dense3)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [10]:
def create_inception_resnet_v2():
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=(width, height, 3), include_top=False, weights='imagenet')(model_input)
            
    model_dense1 = GlobalAveragePooling2D(name="avg_pool")(model_main)
    model_dense1 = BatchNormalization()(model_dense1)

    model_dense2 = tf.keras.layers.Dense(256, activation =  tfa.activations.mish, activity_regularizer=tf.keras.regularizers.l2(1e-5))(model_dense1)
    model_dense2 = BatchNormalization()(model_dense2)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [11]:
def create_efficientnet():
    #model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.efficientnet.EfficientNetB3(input_shape=(width, height, 3), include_top=False, weights='imagenet')
    #model_dense1 = tf.keras.layers.Flatten()(model_main.output)
    
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)
    
    #model_dense1 = tf.keras.layers.Flatten()(layer)

    model_dense2 = tf.keras.layers.Dense(256, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(layer)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [12]:
datagen = create_datagen(inception_resnet_v2_preprocess)
dataloader = create_dataloader(datagen, test_dir, 'test')
inception_resnet_v2 = create_inception_resnet_v2()
inception_resnet_v2.load_weights(f'{model_dir}inception_resnet_v2.h5')

inception_pred = inception_resnet_v2.predict(dataloader, steps=dataloader.samples)

Found 794 images belonging to 1 classes.


2022-04-14 15:35:45.402830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:35:45.490325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:35:45.491083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:35:45.493823: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

219070464/219055592 [==============================] - 3s 0us/step


2022-04-14 15:36:00.955407: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-14 15:36:04.808316: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [13]:
datagen = create_datagen(xception_preprocess)
dataloader = create_dataloader(datagen, test_dir_seg, 'seg_test')
xception = create_xception()
xception.load_weights(f'{model_dir}xception.h5')

xception_pred = xception.predict(dataloader, steps=dataloader.samples)

Found 794 images belonging to 1 classes.
83697664/83683744 [==============================] - 1s 0us/step


In [14]:
datagen = create_datagen(tf.keras.applications.efficientnet.preprocess_input)
dataloader = create_dataloader(datagen, test_dir, 'test')
efficientnet = create_efficientnet()
efficientnet.load_weights(f'{model_dir}efficientnetB3.h5')

efficientnet_pred = efficientnet.predict(dataloader, steps=dataloader.samples)

Found 794 images belonging to 1 classes.
43950080/43941136 [==============================] - 1s 0us/step


In [15]:
combos = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (0.5, 0.5, 0), (0.5, 0, 0.5), (0, 0.5, 0.5), (1/3, 1/3, 1/3), (0.25, 0.5, 0.25), (0.1, 0.5, 0.4), (0.3, 0.5, 0.2), (0.3, 0.6, 0.1), (0.2, 0.6, 0.2), (0.2, 0.5, 0.3), (0.15, 0.5, 0.35), (0.1, 0.4, 0.5), (0.25, 0.4, 0.35), (0.05, 0.55, 0.4)]

for combo in combos:
    w1, w2, w3 = combo
    inception_effnet_pred = w1 * efficientnet_pred + w2 * inception_pred + w3 * xception_pred

    class_list = []

    for i in range(0, inception_effnet_pred.shape[0]):
        y_class = inception_effnet_pred[i, :].argmax(axis=-1)
        class_list += [species_list[y_class]]

    submission = pd.DataFrame()
    submission['file'] = dataloader.filenames
    submission['file'] = submission['file'].str.replace(r'test/', '')
    submission['species'] = class_list

    submission.to_csv(f'efficientnetb3_inceptionresnetv2_xception_{w1}_{w2}_{w3}_submission.csv', index=False)

print('Submission file generated. All done.')

Submission file generated. All done.


In [16]:
np.save('inception_resnet_pred', inception_pred)
np.save('efficientnetb3_pred', efficientnet_pred)
np.save('xception_pred', xception_pred)